# Soh estimation experimentation of Ford vehicles
Based on the file `ford_eda.ipynb` we have seen that we could use the `charging.battery_energy` to estimate the SoH.  

All those result are based on what has been analysed in `ford_eda.ipynb`

We added one filter on the SoC for the SoH calculation, a seocnd one on the charge/discharge could also be added.

In [ ]:
! mkdir -p data_cache/

## Imports

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import linregress as lr

from core.plt_utils import plt_3d_df
from core.stats_utils import lr_params_as_series
from core.pandas_utils import *
from transform.processed_tss.main import get_processed_tss

## Setup

### Data extraction

In [ ]:
tss = get_processed_tss(brand="ford", force_update=True)

## Battery energy EDA
We will use the battery energy to estimate the SoH.  
Let's visualize the battery energy to understand it better.

## Timeseries


In [ ]:
px.box(
    tss,
    x="soc",
    y="battery_energy",
    color="capacity",
)

We can see in the plot above that the battery energy does not start at 0 kwh.  
To appropriatly estimate the SoH as the battery_energy / expected_battery_energy we will first estimate the expected battery energy.  
We will express it as the maximum battery energy recorded at a given SoC.  
We are effectively assuming that the expected battery energy was recorded because at least one of the followed vehicles has a battery SoH of 100% at that SoC.  
This should become more accurate as we will have more vehicles in our dataset.

In [ ]:
max_energy = (
    tss
    .groupby(["capacity", "soc"])
    .agg(
        max_battery_energy=pd.NamedAgg(column="battery_energy", aggfunc="max"),
        max_battery_energy_095=pd.NamedAgg(column="battery_energy", aggfunc=lambda x: x.quantile(0.9))
    )
    .reset_index(drop=False)
)
max_energy

In [ ]:
px.scatter(
    max_energy,
    x="soc",
    y="max_battery_energy_095",
    color="capacity",
)

## SoH estimation

In [ ]:
max_energy = (
    tss
    .pipe(left_merge, max_energy, ["capacity", "soc"], ["capacity", "soc"], ["max_battery_energy", "max_battery_energy_095"])
    .eval("soh = battery_energy / capacity * 100")
    .eval("soh_095 = battery_energy / max_battery_energy_095 * 100")
)
tss

In [ ]:
soh_per_vehicle = (
    tss
    .groupby("vin")
    .agg({
        "soh": "median",
        "soh_095": "median",
        "odometer": "last",
        "model": Series.mode,
        "version": Series.mode,
        "capacity": Series.mode,
    })
    .reset_index(drop=False)
)

In [ ]:
px.scatter(
    max_energy,
    x="soc",
    y="max_battery_energy_095",
    color="capacity",
)

## SoH estimation

In [ ]:
tss:DF = (
    tss
    .pipe(left_merge, max_energy, ["capacity", "soc"], ["capacity", "soc"], ["max_battery_energy", "max_battery_energy_095"])
    .eval("soh = battery_energy / max_battery_energy * 100")
    .eval("soh_095 = battery_energy / max_battery_energy_095 * 100")
)
tss

In [ ]:
soh_per_vehicle = (
    tss
    .groupby("vin")
    .agg({
        "soh": "median",
        "soh_095": "median",
        "odometer": "last",
        "model": Series.mode,
        "version": Series.mode,
        "capacity": Series.mode,
    })
    .reset_index(drop=False)
)
soh_per_vehicle

In [ ]:
fig = px.scatter(
    soh_per_vehicle,
    x="odometer",
    y="soh_095",
    color="capacity",
    height=600,
    title="Average State-of-Health (SoH) vs Mileage",
    trendline="ols",
    trendline_scope="overall",
    hover_data=["vin"]
)
fig.update_layout(
    xaxis_title="Latest mileage (km)",
    yaxis_title="SoH (%)",
    legend_title="Model",
)
fig.update_traces(line=dict(color='black', dash='dash'))

fig.show()

## Conclusion

Given their odometer, the batteries seem very degraded.  
According to this [doc](https://www.lhmfordmesa.com/ford-mustang-mach-e-battery.htm#:~:text=How%20long%20do%20Ford%20Mustang%20Mach%2DE%20batteries%20last%3F) however, it is to be expected that the battery lasts between 3-5 years.  